In [26]:
from pathlib import Path
import subprocess

# -------------------------------------------------------
# 🎯 사용자 지정: 대상 영상 이름 (확장자 제외)
# -------------------------------------------------------
target_video = "M16_visit13_UpperLimb"

# -------------------------------------------------------
# 🧭 경로 설정
# -------------------------------------------------------
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/data/d02/Won_Kim_research_at_Bosanjin/M16")
OutputRoot  = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M16")

# 대상 영상 탐색
video_candidates = list(BASE_DIR.glob(f"{target_video}.*")) 
if not video_candidates:
    raise FileNotFoundError(f"❌ 대상 영상이 없습니다: {target_video}")

Input = video_candidates[0]
OutputDir = OutputRoot / target_video
OutputDir.mkdir(parents=True, exist_ok=True)

print(f"""
📁 디렉토리 구조 설정 완료:
   • Input Video : {Input}
   • OutputDir   : {OutputDir}
""")

# -------------------------------------------------------
# ✂️ 세그먼트 정의 (frame 단위)
# -------------------------------------------------------
segments = [
    (5*30,  54*30, "frontal__biceps_curl__1"),
    (71*30,  116*30, "frontal__biceps_curl__2"),
    (235*30,  268*30, "frontal__shoulder_flexion__1"),
    (289*30,  321*30, "frontal__shoulder_flexion__2"),
    (421*30,  453*30, "frontal__shoulder_press__1"),
    (472*30,  506*30, "frontal__shoulder_press__2"),
    (529*30,  563*30, "frontal__shoulder_abduction__1"),
    (589*30,  627*30, "frontal__shoulder_abduction__2"),

    ]

# -------------------------------------------------------
# 💾 segmentation txt 저장
# -------------------------------------------------------
segment_txt = OutputDir / "segment_frames.txt"
with open(segment_txt, "w", encoding="utf-8") as f:
    f.write("# label, start_frame, end_frame\n")
    for start, end, label in segments:
        f.write(f"{label}, {start}, {end}\n")

print(f"📝 segmentation info saved: {segment_txt}")

# -------------------------------------------------------
# ✂️ FFmpeg 컷팅 함수
# -------------------------------------------------------
def cut_video(input_video, start_frame, end_frame, output_video, fps=30):
    """FFmpeg으로 mp4 구간 자르기 (frame 기반, 정확/빠름/디버깅 포함)"""
    def to_ffmpeg_time(frame_idx):
        t = frame_idx / fps
        m, s = divmod(t, 60)
        return f"{int(m):02d}:{int(s):02d}"

    start = to_ffmpeg_time(start_frame)
    end = to_ffmpeg_time(end_frame)

    cmd = [
        "ffmpeg",
        "-y",
        "-ss", start,
        "-to", end,
        "-i", str(input_video),
        "-c", "copy",
        str(output_video)
    ]

    print(f"[DEBUG] 실행 명령어: {' '.join(cmd)}")
    proc = subprocess.run(cmd, capture_output=True, text=True)

    if proc.returncode != 0:
        print(f"❌ ffmpeg 오류:\n{proc.stderr}")
    else:
        print(f"✅ ffmpeg 성공적으로 실행됨")

    if Path(output_video).exists():
        size = Path(output_video).stat().st_size / 1024
        print(f"📁 파일 생성 확인: {output_video} ({size:.1f} KB)")
    else:
        print(f"⚠️ 파일이 생성되지 않았습니다: {output_video}")

# -------------------------------------------------------
# 🚀 컷팅 실행 (OutputDir에 직접 저장)
# -------------------------------------------------------
for start, end, label in segments:
    output_path = OutputDir / f"{label}.mp4"
    cut_video(Input, start, end, output_path)



📁 디렉토리 구조 설정 완료:
   • Input Video : /workspace/nas203/ds_RehabilitationMedicineData/data/d02/Won_Kim_research_at_Bosanjin/M16/M16_visit13_UpperLimb.MP4
   • OutputDir   : /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M16/M16_visit13_UpperLimb

📝 segmentation info saved: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M16/M16_visit13_UpperLimb/segment_frames.txt
[DEBUG] 실행 명령어: ffmpeg -y -ss 00:05 -to 00:54 -i /workspace/nas203/ds_RehabilitationMedicineData/data/d02/Won_Kim_research_at_Bosanjin/M16/M16_visit13_UpperLimb.MP4 -c copy /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M16/M16_visit13_UpperLimb/frontal__biceps_curl__1.mp4
✅ ffmpeg 성공적으로 실행됨
📁 파일 생성 확인: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M16/M16_visit13_UpperLimb/frontal__biceps_curl__1.mp4 (141400.9 KB)
[DEBUG] 실행 명령어: ffmpeg -y -ss 01:11 -to 01:

In [5]:
from pathlib import Path
import subprocess

# -------------------------------------------------------
# 🎯 사용자 지정
# -------------------------------------------------------
target_video = "M03_VISIT6_LowerLimb"

BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/data/d02/Won_Kim_research_at_Bosanjin/M03")
OutputRoot = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M03")

# -------------------------------------------------------
# 🧭 입력 영상 찾기
# -------------------------------------------------------
video_candidates = list(BASE_DIR.glob(f"{target_video}.*"))
if not video_candidates:
    raise FileNotFoundError(f"❌ 대상 영상이 없습니다: {target_video}")

Input = video_candidates[0]

# 출력 디렉토리 & segment txt
OutputDir = OutputRoot / target_video
segment_txt = OutputDir / "segment_frames.txt"
OutputDir.mkdir(parents=True, exist_ok=True)

if not segment_txt.exists():
    raise FileNotFoundError(f"❌ segment_frames.txt 없음: {segment_txt}")

print(f"""
📁 디렉토리 구조:
   • Input Video : {Input}
   • OutputDir   : {OutputDir}
   • Segments    : {segment_txt}
""")

# -------------------------------------------------------
# 🗑 기존 컷팅 영상 삭제
# -------------------------------------------------------
old_videos = list(OutputDir.glob("*.mp4"))
if old_videos:
    print(f"🗑 기존 영상 {len(old_videos)}개 삭제 중...")
    for v in old_videos:
        v.unlink()
    print("✅ 기존 mp4 모두 삭제 완료")
else:
    print("ℹ️ 기존 mp4 파일 없음")

# -------------------------------------------------------
# 📖 segment txt 읽기
# -------------------------------------------------------
segments = []
with open(segment_txt, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        label, start, end = line.split(",")
        segments.append((int(start), int(end), label.strip()))

print(f"🔍 segment count: {len(segments)}")

# -------------------------------------------------------
# ✂ FFmpeg 컷팅 함수
# -------------------------------------------------------
def cut_video(input_video, start_frame, end_frame, output_video, fps=30):

    def to_ffmpeg_time(frame_idx):
        t = frame_idx / fps
        m, s = divmod(t, 60)
        return f"{int(m):02d}:{int(s):02d}"

    start = to_ffmpeg_time(start_frame)
    end = to_ffmpeg_time(end_frame)

    cmd = [
        "ffmpeg",
        "-y",
        "-ss", start,
        "-to", end,
        "-i", str(input_video),
        "-c", "copy",
        str(output_video)
    ]

    print(f"[DEBUG] 실행 명령어: {' '.join(cmd)}")
    proc = subprocess.run(cmd, capture_output=True, text=True)

    if proc.returncode != 0:
        print(f"❌ ffmpeg 오류:\n{proc.stderr}")
    else:
        print(f"✅ ffmpeg 성공")

    if Path(output_video).exists():
        size = Path(output_video).stat().st_size / 1024
        print(f"📁 생성됨: {output_video} ({size:.1f} KB)")
    else:
        print(f"⚠️ 생성 실패: {output_video}")

# -------------------------------------------------------
# 🚀 컷팅 실행
# -------------------------------------------------------
for start, end, label in segments:
    output_path = OutputDir / f"{label}.mp4"
    cut_video(Input, start, end, output_path)



📁 디렉토리 구조:
   • Input Video : /workspace/nas203/ds_RehabilitationMedicineData/data/d02/Won_Kim_research_at_Bosanjin/M03/M03_VISIT6_LowerLimb.MP4
   • OutputDir   : /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M03/M03_VISIT6_LowerLimb
   • Segments    : /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M03/M03_VISIT6_LowerLimb/segment_frames.txt

🗑 기존 영상 5개 삭제 중...
✅ 기존 mp4 모두 삭제 완료
🔍 segment count: 5
[DEBUG] 실행 명령어: ffmpeg -y -ss 00:07 -to 01:00 -i /workspace/nas203/ds_RehabilitationMedicineData/data/d02/Won_Kim_research_at_Bosanjin/M03/M03_VISIT6_LowerLimb.MP4 -c copy /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M03/M03_VISIT6_LowerLimb/lateral__hooklying_hip_flexion__1.mp4
✅ ffmpeg 성공
📁 생성됨: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/M03/M03_VISIT6_LowerLimb/lateral__hooklying_hip_flexion__1.mp4 (248200.1 KB)

In [16]:
import os
import pandas as pd

csv_path = "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/ASAN_01_Repeatition_Counter/data/metadata.csv"
df = pd.read_csv(csv_path)

# 1) AI_dataset 이 포함된 video_path만 필터링
df_ai = df[df['video_path'].str.contains("long")]

# 2) 파일명만 추출
df_ai['file_name'] = df_ai['video_path'].apply(lambda x: os.path.basename(x))

# 3) 중복 제거된 파일명 리스트
unique_files = df_ai['video_path'].unique()

print("총 파일 개수:", len(unique_files))
for f in unique_files:
    print(f)


총 파일 개수: 2
/workspace/nas203/ds_RehabilitationMedicineData/data/d02/sample_data/ICU_sample_video/frontal__longsitting_knee_flexion__1.MP4
/workspace/nas203/ds_RehabilitationMedicineData/data/d02/sample_data/ward_sample_video/frontal__longsitting_knee_flexion__1.MOV


/tmp/ipykernel_47/2329926069.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ai['file_name'] = df_ai['video_path'].apply(lambda x: os.path.basename(x))


In [20]:
#!/usr/bin/env python3
"""
보산진 연구 데이터 동작 이름 순수 중복 분석 스크립트 (경로 리스트 포함 JSON 저장)
- 순수한 동작 이름별로 해당 파일들의 상대 경로 리스트를 JSON에 저장합니다.
"""

import os
import re
import json
from collections import defaultdict
from typing import Dict, Any, List

# === 1. 설정 및 상수 ===
BOSANJIN_PATH = "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/"
OUTPUT_JSON_FILE = "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/ASAN_01_Repeatition_Counter/bosanjin_pure_motion_with_paths.json"

# === 2. 동작 이름 추출 함수 (변경 없음) ===

def extract_pure_motion_name(filename: str) -> str:
    """
    파일명에서 'frontal__', 'lateral__', '__숫자', 확장자를 모두 제거하고 
    순수한 동작 이름만 추출합니다.
    """
    base_name = os.path.splitext(filename)[0].lower()
    
    match = re.match(r'^(lateral|frontal)__', base_name)
    if match:
        base_name = base_name[len(match.group(0)):]
    
    match = re.search(r'__(\d+)$', base_name)
    if match:
        base_name = base_name[:match.start()]
    
    base_name = base_name.strip('_')
    
    return base_name if base_name else 'unknown_file'


# === 3. 경로 포함 분석 함수 (수정됨) ===

def analyze_motion_names_with_paths(directory: str) -> Dict[str, Any]:
    """
    디렉토리를 탐색하며 순수한 동작 이름별로 해당 파일들의 상대 경로를 집계합니다.
    """
    total_files = 0
    # defaultdict를 사용하여 동작 이름(키)에 경로 리스트(값)를 저장
    motion_paths = defaultdict(list)
    
    print(f"파일 시스템 스캔 및 순수 동작 이름 (경로 포함) 분석 시작: {directory}")
    
    for root, _, files in os.walk(directory):
        for file_name in files:
            if os.path.splitext(file_name)[1].lower() != '.mp4':
                 continue
                 
            total_files += 1
            
            pure_motion_name = extract_pure_motion_name(file_name)
            
            # 파일의 상대 경로를 추출하여 리스트에 추가
            relative_path = os.path.relpath(os.path.join(root, file_name), directory)
            motion_paths[pure_motion_name].append(relative_path)
            
    # 최종 JSON 구조를 위해 Dict로 변환하고 카운트 정보 추가
    results = {
        'total_analyzed_files': total_files,
        'motion_analysis': {}
    }
    
    for motion_name, paths in motion_paths.items():
        results['motion_analysis'][motion_name] = {
            'count': len(paths),
            'paths': paths
        }
        
    return results

# === 4. JSON 저장 함수 (경로 반영) ===

def save_results_to_json(data: Dict[str, Any], output_file: str):
    """분석 결과를 JSON 파일로 저장"""
    try:
        os.makedirs(os.path.dirname(output_file), exist_ok=True) 
        with open(output_file, 'w', encoding='utf-8') as f:
            # JSON 파일에 보기 편하게 들여쓰기(indent=4) 적용
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"✅ JSON 보고서가 '{output_file}'에 성공적으로 저장되었습니다.")
        
        # 콘솔에 JSON 구조 미리보기 출력 (상위 5개 동작만)
        print("\n--- JSON 데이터 구조 미리보기 (경로 포함, 상위 5개 동작) ---")
        
        # 경로 목록이 긴 경우, 미리보기에서 경로를 줄여서 출력
        preview_data = {}
        sorted_motions = sorted(data['motion_analysis'].items(), key=lambda item: item[1]['count'], reverse=True)
        
        for name, details in sorted_motions[:5]:
            preview_paths = details['paths'][:2] # 경로 2개만 미리보기
            if details['count'] > 2:
                preview_paths.append(f"... 외 {details['count'] - 2}개 경로")
                
            preview_data[name] = {
                'count': details['count'],
                'paths': preview_paths
            }
            
        print(json.dumps({'motion_analysis_preview': preview_data}, ensure_ascii=False, indent=4))
        print("------------------------------------------------------------------")
        
    except Exception as e:
        print(f"❌ JSON 파일 저장 실패: {e}")

# === 5. 메인 실행 ===

def main():
    if not os.path.exists(BOSANJIN_PATH):
        print(f"❌ 경로가 존재하지 않습니다: {BOSANJIN_PATH}")
        return
    
    # 1. 경로를 포함한 분석 실행
    analysis_results = analyze_motion_names_with_paths(BOSANJIN_PATH)
    
    # 2. JSON 저장
    save_results_to_json(analysis_results, OUTPUT_JSON_FILE)

if __name__ == "__main__":
    main()

파일 시스템 스캔 및 순수 동작 이름 (경로 포함) 분석 시작: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/Won_Kim_research_at_Bosanjin/
✅ JSON 보고서가 '/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/ASAN_01_Repeatition_Counter/bosanjin_pure_motion_with_paths.json'에 성공적으로 저장되었습니다.

--- JSON 데이터 구조 미리보기 (경로 포함, 상위 5개 동작) ---
{
    "motion_analysis_preview": {
        "slr": {
            "count": 71,
            "paths": [
                "M01/M01_VISIT12/lateral__slr__1.mp4",
                "M01/M01_VISIT12/lateral__slr__2.mp4",
                "... 외 69개 경로"
            ]
        },
        "pec_dec_fly": {
            "count": 56,
            "paths": [
                "M01/M01_VISIT12/frontal__pec_dec_fly__1.mp4",
                "M01/M01_VISIT12/frontal__pec_dec_fly__2.mp4",
                "... 외 54개 경로"
            ]
        },
        "shoulder_press": {
            "count": 54,
            "paths": [
                "M01/M01_VISIT12/frontal__shoulder_press__1.mp4",
          